In [10]:
import pandas as pd
from lxml import etree
import bs4 as bs
import requests
import time
import sys
from selenium import webdriver
import tqdm
import seaborn as sns
from nltk.corpus import stopwords

In [24]:
ticker=[]
for i in namelist:
    data=pd.read_html('https://eoddata.com/stocklist/SGX/'+i+'.htm')[4]
    ticker.append(data)

In [25]:
def get_intro(ticker):
    url = 'https://finance.yahoo.com/quote/'+ticker+'/profile?p='+ticker
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')

    driver = webdriver.Chrome(options=options)

#driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    html = etree.HTML(html)

# page = requests.get(url)
# html = etree.HTML(page.text)

# items = html.xpath('//*[@id="Col1-0-Profile-Proxy"]/section/section[2]/p')
    items = html.xpath(
        '//*[@id="Col1-0-Profile-Proxy"]/section/section[2]/descendant-or-self::text()')
    item_2 = html.xpath(
        '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[2]//text()')
    item_3 = html.xpath(
        '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[4]//text()')
    driver.close()
    return [items[1],item_2[0],item_3[0]]

In [34]:
name = ticker['Code'].apply(lambda x: x+'.SI').values.tolist()

In [36]:
intros=dict()
for ticker in tqdm.tqdm_notebook(name,desc='Companies fetched',unit='company'):
    try:
        intros[ticker]=get_intro(ticker)
    except:
        pass

In [39]:
intro_pd=pd.DataFrame({'company':list(intros.keys()),'intro':[ls[0] for ls in list(intros.values())],'segment':[ls[1] for ls in list(intros.values())],'industry':[ls[2] for ls in list(intros.values())]})
intro_pd['introLen']=intro_pd['intro'].apply(len)
intro_pd[intro_pd['introLen']<400].head()

,company,intro,segment,industry,introLen
2,1A4.SI,"AGV Group Limited, an investment holding compa...",Industrials,Metal Fabrication,359
24,1J7.SI,"Jawala Inc., a forest resource company, focuse...",Basic Materials,Lumber & Wood Production,363
37,41H.SI,"BlackGold Natural Resources Limited, an invest...",Energy,Thermal Coal,373
48,42S.SI,"Astaka Holdings Limited, an investment holding...",Real Estate,Real Estate—Development,226
65,528.SI,"Second Chance Properties Ltd, an investment ho...",Consumer Cyclical,Apparel Retail,382


In [ ]:
intro_pd.to_csv('/Users/mac/OneDrive - National University of Singapore/industry-classification-master/data/STI intro.csv')

In [47]:
import pandas_datareader.data as web
stock_list = intro_pd['company'].values.tolist()
stock_data = web.DataReader(name=stock_list,start='20190101',end='20200430',data_source='yahoo')

In [54]:
stock_data.to_csv('/Users/mac/OneDrive - National University of Singapore/industry-classification-master/data/STI price.csv')

In [3]:
stock_data = pd.read_csv('/Users/mac/OneDrive - National University of Singapore/industry-classification-master/data/STI price.csv',header=None)
stock_data.head()

/opt/anaconda3/envs/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,2

,0,1,2,3,4,5,6,7,8,9,...,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738
0,Attributes,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close,...,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume,Volume
1,Symbols,1A0.SI,1A1.SI,1A4.SI,1B0.SI,1B1.SI,1B6.SI,1C0.SI,1C3.SI,1C5.SI,...,XZL.SI,Y03.SI,Y06.SI,Y35.SI,Y45.SI,Y92.SI,YYB.SI,Z25.SI,Z59.SI,Z74.SI
2,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,0.20000000298023224,0.16373825073242188,0.014999999664723873,0.3149999976158142,0.5681533217430115,0.05400000140070915,0.023000000044703484,1.014137864112854,0.0949999988079071,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-02,0.20000000298023224,0.16373825073242188,0.014999999664723873,0.3149999976158142,0.5634577870368958,0.05400000140070915,0.023000000044703484,1.014137864112854,0.0949999988079071,...,NaN,3700.0,0.0,1100.0,0.0,12819100.0,0.0,552500.0,1944500.0,26096400.0


In [97]:
stock_data.head()

field              Adj Close     Volume
0          value                       
2019-01-02 1B0.SI   0.000000   626000.0
           1B1.SI  -0.008265   170000.0
           1B6.SI   0.000000    15000.0
           1D0.SI  -0.020833   293700.0
           1D3.SI  -0.142857  4092100.0

In [4]:
stock_data = stock_data.set_index(0)
symbols = stock_data.loc['Symbols'].values.tolist()
stock_data.columns = pd.MultiIndex.from_product([['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'],symbols[:int(len(symbols)/6)]],names=['field','value'])
stock_data = stock_data.iloc[3:,:]

In [5]:
stock_data=stock_data.astype(float)
stock_data['Adj Close'] = stock_data['Adj Close'].pct_change()
stock_data = stock_data[['Adj Close','Volume']]
stock_data = stock_data.stack()
stock_data = stock_data.loc[stock_data['Volume']!=0,:]
stock_data.dropna(inplace=True)
stock_data.reset_index(inplace=True)
stock_data.reset_index(drop=True,inplace=True)
stock_data.columns = ['date','company','return','vol']
stock_data = stock_data.iloc[:,:-1]

In [6]:
stock_data.head()

,date,company,return
0,2019-01-02,1B0.SI,0.000000
1,2019-01-02,1B1.SI,-0.008265
2,2019-01-02,1B6.SI,0.000000
3,2019-01-02,1D0.SI,-0.020833
4,2019-01-02,1D3.SI,-0.142857


In [17]:
import keras
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling,lecun_normal,glorot_uniform
from keras.models import Model
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from clustering_class import *
from combine_models import *
from utils import *
from DEC import *
from generate_word_vector import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
path = '/Users/mac/OneDrive - National University of Singapore/industry-classification-master/data'

In [7]:
des = pd.read_csv(path+'/STI intro.csv',index_col=0)

In [11]:
all_data = {}
company_list=des['company'].values
desc_list=des['intro'].values
stopwords = stopwords.words('english')+[' ']
vector_generator=generate_doc_vector(company_list,desc_list,stopwords,0.8,5,'English')
bag_of_words=vector_generator.bag_of_words()
all_data['bag of words']=bag_of_words

In [21]:
all_data['bert'] = pd.read_csv(path+'/STI bert.csv',index_col=0)

In [12]:
for i in range(50,401,50):
    doc2vec_1=vector_generator.doc2vector(vector_size=i,alpha=0.01)
    doc2vec_2=vector_generator.doc2vector(vector_size=i,alpha=0.01,dm=0)
    all_data['pvdm %d length'%i]=doc2vec_1
    all_data['pvdbow %d length'%i]=doc2vec_2

In [13]:
for i in range(200,1101,100):
    Lsi=vector_generator.LSI(num_topics=i)
    all_data['lsi %d length'%i]=Lsi

In [14]:
segment=des.segment
industry=des.industry
print('%d segments'%len(set(segment)))
print('%d industries'%len(set(industry)))

11 segments
110 industries


In [22]:
print(all_data.keys())

dict_keys(['bag of words', 'pvdm 50 length', 'pvdbow 50 length', 'pvdm 100 length', 'pvdbow 100 length', 'pvdm 150 length', 'pvdbow 150 length', 'pvdm 200 length', 'pvdbow 200 length', 'pvdm 250 length', 'pvdbow 250 length', 'pvdm 300 length', 'pvdbow 300 length', 'pvdm 350 length', 'pvdbow 350 length', 'pvdm 400 length', 'pvdbow 400 length', 'lsi 200 length', 'lsi 300 length', 'lsi 400 length', 'lsi 500 length', 'lsi 600 length', 'lsi 700 length', 'lsi 800 length', 'lsi 900 length', 'lsi 1000 length', 'lsi 1100 length', 'bert'])


In [15]:
def lr_sch(epoch):
    return 1/(epoch+1)

In [18]:
tol = 0.001
maxiter = 1000
update_interval = 100
n_clusters = 12
n_epochs=150
batch_size_auto=128
batch_size=64
init = glorot_uniform()
pretrain_optimizer = SGD(lr=1, momentum=0.9)
optimizer=SGD(lr=0.01,momentum=0.9,decay=0.01/n_epochs)
lr_scheduler=keras.callbacks.LearningRateScheduler(lr_sch, verbose=0)
early_stopping= keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0002, patience=5)
logdir="logs/fit/USBert" 
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
output=[]
kmeans = KMeans(n_clusters=n_clusters)

In [23]:
for key,input in all_data.items():
    input_x=input.iloc[:,1:]
    dims=[input_x.shape[-1],500,200,2000,50]
    autoencode, encoder = autoencoder(dims, act='relu', init=init, dropout=0.2)
    autoencode.compile(optimizer=pretrain_optimizer, loss='mse')
    autoencode.fit(input_x, input_x, batch_size=batch_size, epochs=n_epochs,callbacks=[lr_scheduler,early_stopping])
    kmeans=kmeans.fit(encoder.predict(input_x))
    y_init=kmeans.cluster_centers_
    loss = 0
    index = 0
    index_array = np.arange(input_x.shape[0])
    clustering_layer = ClusteringLayer(n_clusters=n_clusters, name='clustering',weights=[y_init])(encoder.output)
    model = Model(inputs=encoder.input, outputs=clustering_layer)    
    model.compile(loss='kld', optimizer=optimizer)
    y_pred_last=kmeans.predict(encoder.predict(input_x))
    for ite in range(maxiter):
        if ite % update_interval == 0:
            q = model.predict(input_x, verbose=0)
            p = target_distribution(q)  # update the auxiliary target distribution p

            # evaluate the clustering performance
            y_pred = q.argmax(1)

            # check stop criterion
            delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
            y_pred_last = np.copy(y_pred)
            print('the '+str(ite)+'th training on '+key)
            if ite > 0 and delta_label < tol:
                print('delta_label = ', delta_label, ' < %d'%tol)
                print('Reached tolerance threshold. Stopping training.')
                break
        idx = index_array[index * batch_size: min((index+1) * batch_size, input_x.shape[0])]
        loss = model.train_on_batch(x=input_x.iloc[idx], y=p[idx])
        index = index + 1 if (index + 1) * batch_size <= input_x.shape[0] else 0
    q = model.predict(input_x,verbose=0)
    clas=pd.DataFrame(np.argmax(q,1),columns=[key])
    clas.insert(0,'company',input['company'])
    output.append(clas)



Epoch 1/150
623/623 [==============================] - 2s 3ms/step - loss: 0.1098
Epoch 2/150
320/623 [==============>...............] - ETA: 0s - loss: 0.0742

/opt/anaconda3/envs/venv/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


623/623 [==============================] - 0s 562us/step - loss: 0.0659
Epoch 3/150
623/623 [==============================] - 0s 432us/step - loss: 0.0365
Epoch 4/150
623/623 [==============================] - 0s 453us/step - loss: 0.0248
Epoch 5/150
623/623 [==============================] - 0s 578us/step - loss: 0.0194
Epoch 6/150
623/623 [==============================] - 0s 417us/step - loss: 0.0167
Epoch 7/150
623/623 [==============================] - 0s 450us/step - loss: 0.0142
Epoch 8/150
623/623 [==============================] - 0s 428us/step - loss: 0.0129
Epoch 9/150
623/623 [==============================] - 0s 417us/step - loss: 0.0119
Epoch 10/150
623/623 [==============================] - 0s 425us/step - loss: 0.0107
Epoch 11/150
623/623 [==============================] - 0s 414us/step - loss: 0.0101
Epoch 12/150
623/623 [==============================] - 0s 426us/step - loss: 0.0096
Epoch 13/150
623/623 [==============================] - 0s 452us/step - loss: 0.0088


/opt/anaconda3/envs/venv/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


623/623 [==============================] - 0s 256us/step - loss: 0.0233
Epoch 4/150
623/623 [==============================] - 0s 254us/step - loss: 0.0120
Epoch 5/150
623/623 [==============================] - 0s 259us/step - loss: 0.0071
Epoch 6/150
623/623 [==============================] - 0s 277us/step - loss: 0.0054
Epoch 7/150
623/623 [==============================] - 0s 273us/step - loss: 0.0044
Epoch 8/150
623/623 [==============================] - 0s 414us/step - loss: 0.0038
Epoch 9/150
623/623 [==============================] - 0s 277us/step - loss: 0.0034
Epoch 10/150
623/623 [==============================] - 0s 292us/step - loss: 0.0033
Epoch 11/150
623/623 [==============================] - 0s 369us/step - loss: 0.0035
Epoch 12/150
623/623 [==============================] - 0s 450us/step - loss: 0.0031
Epoch 13/150
623/623 [==============================] - 0s 387us/step - loss: 0.0029
Epoch 14/150
623/623 [==============================] - 0s 316us/step - loss: 0.0029

...

/opt/anaconda3/envs/venv/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


623/623 [==============================] - 0s 371us/step - loss: 0.0679
Epoch 3/150
623/623 [==============================] - 0s 367us/step - loss: 0.0356
Epoch 4/150
623/623 [==============================] - 0s 371us/step - loss: 0.0243
Epoch 5/150
623/623 [==============================] - 0s 371us/step - loss: 0.0193
Epoch 6/150
623/623 [==============================] - 0s 368us/step - loss: 0.0163
Epoch 7/150
623/623 [==============================] - 0s 368us/step - loss: 0.0140
Epoch 8/150
623/623 [==============================] - 0s 435us/step - loss: 0.0125
Epoch 9/150
623/623 [==============================] - 0s 379us/step - loss: 0.0116
Epoch 10/150
623/623 [==============================] - 0s 387us/step - loss: 0.0108
Epoch 11/150
623/623 [==============================] - 0s 360us/step - loss: 0.0100
Epoch 12/150
623/623 [==============================] - 0s 354us/step - loss: 0.0097
Epoch 13/150
623/623 [==============================] - 0s 352us/step - loss: 0.0090


/opt/anaconda3/envs/venv/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


623/623 [==============================] - 0s 384us/step - loss: 0.1009
Epoch 3/150
623/623 [==============================] - 0s 389us/step - loss: 0.0835
Epoch 4/150
623/623 [==============================] - 0s 361us/step - loss: 0.0644
Epoch 5/150
623/623 [==============================] - 0s 347us/step - loss: 0.0615
Epoch 6/150
623/623 [==============================] - 0s 346us/step - loss: 0.0551
Epoch 7/150
623/623 [==============================] - 0s 344us/step - loss: 0.0527
Epoch 8/150
623/623 [==============================] - 0s 340us/step - loss: 0.0521
Epoch 9/150
623/623 [==============================] - 0s 344us/step - loss: 0.0489
Epoch 10/150
623/623 [==============================] - 0s 349us/step - loss: 0.0506
Epoch 11/150
623/623 [==============================] - 0s 392us/step - loss: 0.0481
Epoch 12/150
623/623 [==============================] - 0s 387us/step - loss: 0.0485
Epoch 13/150
623/623 [==============================] - 0s 349us/step - loss: 0.0487


KeyError: 'company'

In [24]:
from sklearn.preprocessing import LabelEncoder
official_df=des[['company','segment']]
le=LabelEncoder()
le_class=le.fit_transform(official_df['segment'])
#word_class=official_df['official_class']
official_df['segment']=le_class
combiner=combine_models()
all_df=[official_df]+output
all_cluster=combiner.get_combine_cluster(all_df)
all_cluster.head()

/opt/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,company,segment,bag of words,pvdm 50 length,pvdbow 50 length,pvdm 100 length,pvdbow 100 length,pvdm 150 length,pvdbow 150 length,pvdm 200 length,...,lsi 200 length,lsi 300 length,lsi 400 length,lsi 500 length,lsi 600 length,lsi 700 length,lsi 800 length,lsi 900 length,lsi 1000 length,lsi 1100 length
0,1A0.SI,2,9,1,8,8,9,11,9,7,...,8,2,4,5,6,6,3,1,4,9
1,1A1.SI,7,3,6,8,1,9,6,9,3,...,10,10,1,10,8,6,6,0,9,9
2,1A4.SI,7,9,8,7,3,5,8,7,4,...,9,3,8,10,10,9,11,0,9,7
3,1B0.SI,1,10,0,8,0,9,5,9,9,...,5,10,4,8,10,5,8,10,9,0
4,1B1.SI,6,4,8,8,3,9,2,9,7,...,4,10,8,10,10,3,8,6,3,9


In [25]:
second_analysis=performance_analysis(mode='return_performance',class_df=all_cluster,return_df=stock_data)
second_analysis.get_wellness()

/Users/mac/OneDrive - National University of Singapore/industry-classification-master/combine_models.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.return_map[name][cl].dropna(1,'all',inplace=True)
/Users/mac/OneDrive - National University of Singapore/industry-classification-master/combine_models.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.return_map[name][cl]['return']=self.return_map[name][cl].mean(1)
/opt/anaconda3/envs/venv/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountere

In [26]:
for name,result in second_analysis.R2:
    print('Mean of R square of {} classification is: {}'.format(name, result))

Mean of R square of segment classification is: 0.17478746377564064
Mean of R square of bag of words classification is: 0.17536400995394605
Mean of R square of pvdm 50 length classification is: 0.2154768628209767
Mean of R square of pvdbow 50 length classification is: 0.25989906238078486
Mean of R square of pvdm 100 length classification is: 0.21333919138458954
Mean of R square of pvdbow 100 length classification is: 0.2973027645603855
Mean of R square of pvdm 150 length classification is: 0.21843969204044789
Mean of R square of pvdbow 150 length classification is: 0.1292028477733238
Mean of R square of pvdm 200 length classification is: 0.211397604321254
Mean of R square of pvdbow 200 length classification is: 0.23922089112814443
Mean of R square of pvdm 250 length classification is: 0.22416348266448255
Mean of R square of pvdbow 250 length classification is: 0.13778706998929252
Mean of R square of pvdm 300 length classification is: 0.203494195420326
Mean of R square of pvdbow 300 leng